## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 1. Algorithms: A Glimpse of CGT

![Trees](images/trees.jpg)

## Algorithms

A good algorithm is short yet powerful.

Like this one-line recursive implementation of the [Euclidean Algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm):

In [ ]:
gcd(a, b) =  b == 0 ?  a  :  gcd(b, a % b)

In [ ]:
gcd(237934564903214447864796765423105634076804062380125, 46805092789549878742860339647868490386740234760875)

This code is directly translated from the corresponding mathematical formula:
$$
\gcd(a, b) = \begin{cases}
a, & \text{if } b = 0,\\
\gcd(b, a \bmod b), & \text{else,}
\end{cases}
$$
which is helpful for **proving desirable properties** of an algorithm (meaning, termination, correctness, ...).  Also it can serve as a **model for variations** (for speed, ...) or **extensions** (progress reports, additional data, ...).

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">

* A **simple function** can be defined by just assigning a formula to the function call expression, like
  ```julia
  square(x) = x^2
  ```
* Then `square(2)` should return `4`.

</div>

Many of my algorithms are variations or extensions of the **orbit algorithm**, a simple $3$-step procedure of the form:

1. **Initialize.** (a list)
2. **Loop.**  (over the list and grow it)
3. **Return.**  (the list)

We will soon see that the list really is a **tree**, and that the algorithm performs some kind of **tree traversal** ...

##  Tree Traversal: BFS vs DFS

* A **tree** is a connected, cycle-free (simple) graph.  A **rooted tree** is a tree with a distinguished node, the **root** of the tree.
* With respect to the root, each (non-root) node in the tree has a unique **parent** node, of which it is a **child** node.
* Implicitly, a rooted tree is then a **digraph** (i.e., a directed graph) with arrows pointing away from the root, from the parent to its children.

<div class="alert alert-block alert-success">

* **Example.** Here is tree with node set $\{1,2, \ldots, 7\}$ and root $6$ (in white):

![e7 tree](images/e7tree.png)
![e7 direct](images/e7direct.png)

The parent of node $2$ is node $4$.  The children of node $4$ are nodes $2$ and $3$.
</div>

* **Breath First Search** (BFS) and **Depth First Search**  (DFS) provide strategies for traversing a tree in a simple and systematic fashion.
* For this, we assume that each **node** $x$ in the tree knows all its **children** as a list $x$.next.
* BFS uses a FIFO **queue** (first in, first out), while DFS uses a LIFO **stack** (last in, first out).
* We use `pop()` to remove an item, and `push()` to add items to either data structure.
* Then the two **algorithms** for visiting all nodes of a tree with root $x$ can be described as an initialization step, followed by a loop.

![bfs vs dfs](images/bfsdfs.png)

* In practice, BFS can take advantage of **dynamic lists** in a `for` loop.
* While DFS, using the **function call stack**, can be implemented **recursively**.

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">  

* A **for loop** has the general form
  ```julia
  for <var> in <list>
      <body>
  end
  ```
* A **function definition** has the general form
  ```julia
  function <name> ( <params> )
      <body>
  end
  ```

</div>

* Here is an implementation of BFS in `julia`, with a `visit` function as explicit argument.

In [ ]:
function BFS(x, visit)
    Q = [x]
    for y in Q
        visit(y)
        append!(Q, y.next)
    end
end

* Here is an implementation of DFS in `julia`, with a `visit` function as explicit argument.

In [ ]:
function DFS(x, visit)
    visit(x)
    for z in x.next
        DFS(z, visit)
    end
end

* For an application to the above example of a tree let's organize the tree as a collection of `Node` records, with a component `name` (for their own name, e.g., their position in a list `nodes`) and a component `next` (a list of references to their children).

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">

* Users can define their own **data types**.
* A new **composite type definition** has the form
  ```julia
  struct <Name>
      <parts>
  end
  ```
* Each part, in addition to its **name**, can declare its **type** after `::`
* The name of the data type serves as **default constructor**, which when called creates a new object of this type, with given values assigned to each of the parts. 
</div>

In [ ]:
struct Node
    name
    next::Array{Node}
end

* A single node with no children can then be made with `julia`'s implicitly provided constructor.

In [ ]:
Node(9, [])

* The actual tree can then be created from a list of parents as follows.

In [ ]:
parents = [3,4,4,5,6,6,6]
nodes = [Node(i, []) for i in 1:7]
for (i,k) in pairs(parents)
    i == k || push!(nodes[k].next, nodes[i])
end
root = nodes[6]

* Define a visitor function `pr` that simply prints the name of each node it encounters ...

In [ ]:
pr(x) = print(x.name, ", ")

* ... BFS and DFS list all the nodes in slightly different orders:

In [ ]:
BFS(root, pr);

In [ ]:
DFS(root, pr);

* Recall the graph: ![e7 tree](images/e7tree.png)

* The recursive strategy can be modified to suit a specific purpose, e.g., to print a tree as a tree 

In [ ]:
function tree_print(x, indent = "", first = true)
    first || print("\n", indent)
    print("-", x.name);
    first = true
    for c in x.next
        tree_print(c, indent * "  ", first)
        first = false
    end
end

In [ ]:
tree_print(root);

### Graph Traversal

* Both BFS and DFS can be applied to a (simple or directed) **graph**.
* Here, the **same node** then can possibly be reached through **different paths**.
* Thus, some care needs to be taken to **manage** these **repeat encounters**.
* We'll see how this is done in the examples below.

###  Applications

* distance between nodes
* shortest paths
* connected components
* ...
* see Exercises.

## Orbit Algorithms

<div class="alert alert-danger">

**Definition.** An **action** (from the right) of a group $G$ on a set $X$ is map $X \times G \to X$ such that
1. $x.1 = x$ for all $x \in X$,
2. $x.(ab) = (x.a).b$ for all $x \in X$ and all $a, b \in G$.

</div>

* **Group actions** are a rich source of graphs.
* Here, the **nodes** of the graph are the elements $x$ of the domain $X$ that is acted upon.
* The **edges** of the graph are of the form $x \stackrel{s}{\longrightarrow} x.s$, implicitly given by the action.
* The elements $s$ typically come from a set of **generators** of the acting group.

<div class="alert alert-danger">

**Definition.**
Let $G$ be a group acting on a set $X$ via $(x, a) \mapsto x.a$, and suppose that $G = \langle S \rangle$ for 
some $S \subseteq G$. The **action graph** $\Gamma(S, X)$ is the (directed) graph with 
* **vertices:** $x \in X$, and
* **edges:** $x \stackrel{s}{\longrightarrow} x.s$ for $x \in X$, $s \in S$.
</div>

* The **orbit** of $x \in X$ under $G$ is the set $x^G = \{x.a : a \in G\}$.  It corresponds to the **connected component** of $x$ in the action graph $\Gamma(S, X)$.
* We formulate and apply a variant of BFS called **orbit algorithm** to compute orbits and other properties of actions of $G$.
* You'll be amazed by what all qualifies as an orbit ...

### The Orbit Algorithm

* In order to have some groups to play with, provide a list of transpositions of adjacent points.

In [ ]:
push!(LOAD_PATH, "./julia")
using permutation

In [ ]:
transpositions(n::Integer) = [transposition(n, j-1, j) for j in 2:n]

* For example, on $4$ points, the **simple transpositions** are $(1,2) = (^{1234}_{2134})$, $(2,3) = (^{1234}_{1324})$ and $(3,4) = (^{1234}_{1243})$. (Which group do they generate?)
* In the calculations below, we will use **one-line notation** $[2,1,3,4]$, $[1,3,2,4]$ and $[1,2,4,3]$ for these.

In [ ]:
n = 4
id = one(Perm, n)

In [ ]:
swaps = transpositions(n)

* Now suppose a finite group $G$ acts on a finite set $X$.
* For a given element $x \in X$, we wish to determine its orbit $x^G$ as the set of all the nodes in the graph $\Gamma(S, X)$ that can be reached from $x$ through the edges labelled by elements $s \in S$.
* This looks like a case for BFS with $y$.next ${} = \{y.s : s \in S\}$.

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">

* **Functions** are **first class objects**:
    * they can be passed as **arguments** into function calls and
    * they can be **returned as values** from function calls.

</div>

* The action of $G$ on $X$ is described by an **action function** `under`: calling `under(x, s)` returns $x.s$ (&ldquo;$x$ under $s$&rdquo;).
* For example, the **standard action** on points $(x, a) \mapsto x^a$ can be expressed as

In [ ]:
onPoints(x, a) = x^a

Then $2^{(2,3)} = 3$:

In [ ]:
onPoints(2, swaps[2])

<div class="alert alert-info">
    
**Orbit Algorithm**
    
* **Input:** a list `aaa` of generating operators, a point `x` of the domain $X$, and an action function `under`. 
* **Output:** the **orbit** $x^G = \{x.a : a \in G\}$ of the point `x` under the action of the group $G$ generated by `aaa`.
    
</div>

In [ ]:
function orbit(aaa, x, under)
    list = [x]
    for y in list
        for a in aaa
            z = under(y, a)
            z in list || push!(list, z)
        end
    end
    return list
end

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">

* The logical operators `||` (OR) and `&&` (AND) are [short-circuiting](https://docs.julialang.org/en/v1/manual/control-flow/#Short-Circuit-Evaluation) and can be used to **abbreviate conditional statements**.

</div>

* **Example.** To find the **orbit** of a point $x$ under the group generated by the swaps: apply the orbit algorithm to 
  - (i) the swaps, 
  - (ii) the point $x$, 
  - (iii) the standard action **on points** $(x, a) \mapsto x^a$

In [ ]:
orbit(swaps, 2, onPoints)

### Elements

* The group $G$ acts on itself by **right multiplication**: $(x, a) \mapsto xa$ is a map from $G \times G$ to $G$ that satisfies the action axioms.
* The set $G$ is the orbit of any element $x \in G$ under this action, in particular of the identity permutation.

In [ ]:
onRight(x, a) = x * a

* To find the **elements** of the group generated by the swaps: apply the orbit algorithm to 
  - (i) the swaps, 
  - (ii) the identity permutation, 
  - (iii) the action **on the right** $(x, a) \mapsto x a$

In [ ]:
orbit(swaps, id, onRight)

* In CGT, it is customary to represent a group $G = \langle A \rangle$ by a **list $A$ of generators** (avoiding the need to list all its elements where possible).
* Let's follow this **design principle** and turn the list of generators into a group object, and from now on formulate algorithms in terms of the group.
* For this, in `julia` we declare a new **composite type** `PermGp` for objects that consist of a list `gens` of permutations (the **generators** of a permutation group), and a single permutation `one` (the **identity element** of that group).

In [ ]:
struct PermGp
    gens::Array{Perm}
    one::Perm
end

In [ ]:
group = PermGp(swaps, id)
group.gens

* Then we can define, for instance, a function `elements` to compute the **elements of a group**:

In [ ]:
elements(group::PermGp) = sort(orbit(group.gens, group.one, onRight))

* test it

In [ ]:
eee = elements(group)

* We can use this function to **test membership** in a group

In [ ]:
import Base.in
in(a::Perm, group::PermGp) = a in elements(group)

In [ ]:
Perm([4,3,2,1]) in group

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">  

* [in](https://docs.julialang.org/en/v1/base/collections/#Base.in)

</div>

### Words in the generators

* Sometimes the question arises: how is a certain group element a **product of the generators**?
* In order to express an element of the group as **word** in the generators, we introduce an action on words.  
* For this, the generators $s \in S$ will be represented by symbols in $\mathbb{N} = \{1,2,3,\dotsc\}$ (Who is acting? On what?)
* The image of a word $w$ over $\mathbb{N}$ under an element $s$ of $\mathbb{N}$ is simply $ws$, the word obtained from $w$ by appending the letter $s$.

In [ ]:
onWords(word, s) = [word; s]

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">  

* [vcat](https://docs.julialang.org/en/v1/base/arrays/#Base.vcat)

</div>

* The corresponding orbit algorithm is now required to produce **two lists in parallel**: the list `list` of elements as before, and a list `words` of corresponding words.
* We now need more control over the lists and make the indices `i` (in `list`) and `k` (in `aaa`) explict.
* As a consequence, the loop over `list` becomes a `while` loop, and we must not forget to increment `i`.

In [ ]:
function orbit_with_words(aaa, x, under)
    list = [x]
    words = Array{Int}[[]]
    for (i, y) in enumerate(list)
        for (k, a) in enumerate(aaa)
            z = under(y, a)
            w = onWords(words[i], k)
            z in list || (push!(list, z), push!(words, w))
        end
    end
    return (list = list, words = words)
end

In [ ]:
orbit_with_words(swaps, id, onRight).words

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">  

* [named tuples](https://docs.julialang.org/en/v1/manual/functions/#Named-Tuples)

</div>

* We now can use this new variant of the orbit algorithm in the same way as before.

In [ ]:
www = orbit_with_words(swaps, id, onRight).words

* Note that, by construction, the orbit algorithm (being a form of BFS) finds a **word of shortest possible length** for each group element.
* In the above example, we can see how the group elements are enumerated by length, as a list that ends with a **unique word of longest length**.  Not every group might have such a unique longest element.

## Conjugacy Classes

* The group $G$ acts on itself by **conjugation**: $(x, a) \mapsto x^a = a^{-1} x a$.
* This action partitions the group $G$ into **conjugacy classes**.
* With suitable arguments, the action function `onPoints` from above also applies to the conjugation action of $G$ on itself.
* Hence, we can use the orbit algorithm to compute the conjugacy class of a single element in $G$.
* Perhaps surprisingly, it also finds the list of **all conjugacy classes** of $G$.

###  Conjugacy Class

* To find the **conjugacy class** of an element $x$ of a group: apply the orbit algorithm to 
  - (i) the generators of the group, 
  - (ii) the element $x$, 
  - (iii) the on-points-action $(x, a) \mapsto x^a = a^{-1} x a$

In [ ]:
class(group::PermGp, x::Perm) = orbit(group.gens, x, onPoints)

* test on a random element

In [ ]:
a = rand(eee)

In [ ]:
c = class(group, a)

In [ ]:
[cycles(a) for a in c]

In [ ]:
print([shape(a) for a in c])

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">  

* [list comprehension](https://docs.julialang.org/en/v1/manual/arrays/#man-comprehensions)

</div>

* Sometimes it is convenient to treat the orbit $x^G$ of the point $x$ under the group $G$ as a pair $(G, x^G)$, where $x^G$ is a set, or a sorted list.
* Let's provide a data type `Orbit` for this.

In [ ]:
struct Orbit
    group
    elts  # sorted!
end

* The class of `a` in `G` is denoted (and constructed) as `a^G`.

In [ ]:
import Base: ^
^(a::Perm, group::PermGp) = Orbit(group, sort(class(group, a)))

In [ ]:
cl = a^group

* We want to be able to test membership in, and to compare orbits: as sorted lists of elements, two classes are the same if their first elements are the same.

In [ ]:
import Base: in, isless, ==
in(x, o::Orbit) = x in o.elts
==(o::Orbit, other::Orbit) = o.elts[1] == other.elts[1]
isless(o::Orbit, other::Orbit) = o.elts[1] < other.elts[1]

In [ ]:
cl == a^group

In [ ]:
cl == (a^0)^group

###  Conjugacy Classes

* To find all conjugacy classes of a group, the list of generators itself needs to be **closed under conjugation**.
* We need a version `orbitx` of the orbit algorithm that **initializes** its queue with **several points**, not just one.

In [ ]:
function orbitx(aaa, xxx, under)
    list = copy(xxx)
    for y in list
        for a in aaa
            z = under(y, a)
            z in list || push!(list, z)
        end
    end
    return list
end

* For example, all conjugates of the swaps (what's this set usually called?)

In [ ]:
o = orbitx(swaps, swaps, onPoints)

In [ ]:
[cycles(a) for a in o]

* Now, in order to find all conjugacy classes of the group $G$, we consider the following action of $G$ on its conjugacy classes.
* The image of a conjugacy class $x^G$ under right multiplication with a group element $a$ is the conjugacy class $(xa)^G$.  (In what sense is this an action? Is it even well-defined?)

In [ ]:
onClasses(x, a) = (x.elts[1] * a)^(x.group)

* To find all **conjugacy classes** of a group $G$:
  - we close the set `gens` of generators of $G$ under conjugation, and
  - compute the orbit of the class $1^G$ of the identity under the above action on classes.

In [ ]:
function conjClasses(gp::PermGp)
    orbit(orbitx(gp.gens, gp.gens, onPoints), (gp.one)^gp, onClasses)
end

In [ ]:
cc = conjClasses(group)

* Let's check that the sizes of the conjugacy classes add up to the size of the group.

In [ ]:
import Base: size
size(o::Orbit) = length(o.elts)

In [ ]:
[size(c) for c in cc]

In [ ]:
n = 5
cc = conjClasses(PermGp(transpositions(n), Perm(n)))
sum(size.(cc))

In [ ]:
[shape(c.elts[1]) for c in cc]

## Subgroups

* Let's try and enumerate all subgroups of $G$ as an orbit ...

<div class="alert alert-success">

**Remarks**

* The operators `aaa` need not be invertible, nor do they need to generated a group: the orbit algorithm does not require the use of inverses.
* Neither do they need to generate a finite domain: the orbit algorithm can terminate if the list `aaa` and the orbit are finite.

</div>

* Sometimes, the operators `aaa` generate a **monoid**.  A **monoid action** is defined in the same way as a group action.
* A well-known monoid is the **power set** $2^S$ of a finite set $S$, with **set union** as its binary operation, generated by the **singleton sets** $\{s\}$, $s \in S$.

In [ ]:
==(group::PermGp, other::PermGp) = elements(group) == elements(other)
isless(group::PermGp, other::PermGp) = elements(group) < elements(other)

* Let $G$ be a group.
* The power set $(2^G, \cup)$ is a monoid, generated by the singletons $\{a\}$, $a \in G$, as atoms.
* $2^G$ acts on the subgroups $H$ of $G$ via **closure**: $H.A = \langle H, A \rangle$.  (Check!)

In [ ]:
closure(group::PermGp, a::Perm) = PermGp(union(group.gens, a), group.one)

In [ ]:
onGroups(x, a) = closure(x, a)

* In fact, each subgroup of $G$ lies in the $2^G$-orbit of the trivial subgroup.  (Check!)

In [ ]:
subgroups(group) = orbit(elements(group), PermGp([], group.one), onGroups)

* Let's apply this to our group, generated by the swaps.

In [ ]:
size(group::PermGp) = length(elements(group))

In [ ]:
size(group)

In [ ]:
subs = subgroups(group)

In [ ]:
length(subs)

In [ ]:
print(size.(subs))

### Conjugacy Classes of Subgroups

* The group $G$ acts on its set of subgroups by conjugation: $(H, a) \mapsto H^a = \{h^a : h \in H\}$.
* Let's implement this as a julia operator `^`, together with a function `subgpClass` that computes the conjugacy class $H^G$ of a given subgroup $H$ of $G$ as $G$-orbit under this action.
* We also define `H^G` as an `Orbit` object.

In [ ]:
^(group::PermGp, a::Perm) = PermGp([x^a for x in group.gens], group.one)
subgpClass(gp::PermGp, subgp::PermGp) = orbit(gp.gens, subgp, onPoints)
^(subgp::PermGp, group::PermGp) = Orbit(group, sort(subgpClass(group, subgp)))

* Combining ideas from above we now define the image of a conjugacy class of subgroups $H^G$ under a singleton $\{a\}$  as the conjugacy class $\langle H, a \rangle^G$. (Is this a well-defined action?)

In [ ]:
onSubgpClasses(x, a) = onGroups(x.elts[1], a)^(x.group)

* To compute all **conjugacy classes of subgroups** of $G$, we determine the orbit of the class of the trivial subgroup under the monoid $(2^G, \cup)$ with respect to that action.

In [ ]:
function subgpClasses(gp::PermGp)
    orbit(elements(gp), PermGp([], gp.one)^gp, onSubgpClasses)
end

In [ ]:
ccs = subgpClasses(group);

In [ ]:
length(ccs)

In [ ]:
print(size.(ccs))

In [ ]:
print([size(c.elts[1]) for c in ccs])

In [ ]:
sum(size.(ccs))

* And a slightly bigger example:

In [ ]:
ccs = subgpClasses(PermGp(transpositions(5), one(Perm, 5)));
length(ccs)

In [ ]:
sum(size.(ccs))

<div class="alert alert-success">

**Remarks.**

* This naive application of the orbit algorithm is obviously not suitable for larger examples.  But it can still serve as high level description for more refined implementations.
* For more counting results on the subgroups of the symmetric group see sequences [A000638](https://oeis.org/A000638) and [A005432](https://oeis.org/A005432) of the **Online Encyclopedia of Integer Sequences**.

## Stabilizer and Transversal

* The performance of certain orbit calculations can be drastically improved by making systematic use of stabilizer subgroups. 

<div class="alert alert-danger">

**Definition.**  Let $G$ be a finite group acting on a set $X$.  The **stabilizer** of the point $x \in X$ is the set $G_x = \{a \in G : x.a = x\}$.  Clearly, $G_x$ is a subgroup of $G$.

</div>

* We can use a variant of the orbit algorithm to determine (and remember), for each point $y$ in the $G$-orbit of $x$, a representative element $t_y \in G$ with $x.t_y = y$.
* This list of representatives will form a **transversal** of the cosets of the stabilizer of $x$ in $G$.
* By Schreier's Theorem, a generating set for the **stabilizer** can be computed from the transversal.

### Transversal

* We initialize an additional list `reps` with the identity element, mapping $x$ to itself: $x.1 = x$
* Whenever a new point $z \in X$ is found, as image under $a \in S$ of a point $y \in X$, we add $t_z := t_y a \in G$ to the list `reps`.

In [ ]:
function orbit_with_transversal(aaa, x, under)
    list = [x]
    reps = [aaa[1]^0]
    for (i, y) in enumerate(list)
        for a in aaa
            z = under(y, a)
            t = reps[i] * a
            z in list || (push!(list, z), push!(reps, t))
        end
    end
    return (list = list, reps = reps)
end

In [ ]:
swaps = transpositions(5)
transversal = orbit_with_transversal(swaps, 2, onPoints)

### Stabilizer

<div class="alert alert-danger">

**Schreier's Theorem.**
Suppose a group $G$, generated by a set $S$,
acts on a set $X$ and that $\{t_y : y \in x^G\}$ is a transversal
of the orbit of $x \in X$.  Then the set
$$
    \{t_y a t_{y.a}^{-1}: a \in A,\, y \in x^G\}
$$
generates the stabilizer $G_x$ of $x$ in $G$.
    
</div>

* In the next variant of the orbit algorithm, we also collect the Schreier generators $t_ya t_{y.a}^{-1}$ in a list `stab`.
* For each newly constructed point $z := y.a$, the simple membership test `z in list` needs to be replaced by  `findfirst(==(z), list)` in order to determine its **exact position** in `list`.
* `findfirst` returns `nothing` if the element `z` is not found in `list`.

In [ ]:
function orbit_with_stabilizer(aaa, x, under)
    list = [x]
    reps = [aaa[1]^0]
    stab = Perm[]
    for (i, y) in enumerate(list)
        for a in aaa
            z = under(y, a)
            l = findfirst(==(z), list) # index of z in list
            if isnothing(l)
                push!(list, z)
                push!(reps, reps[i] * a)
            else   # x^(reps[i] * a) = x^reps[l]
                push!(stab, reps[i] * a / reps[l])
            end
        end
    end
    return (list = list, reps = reps, stab = stab)
end

In [ ]:
stabilizer = orbit_with_stabilizer(swaps, 2, onPoints)

In [ ]:
size(PermGp(stabilizer.stab, swaps[1]^0))

In [ ]:
length(stabilizer.stab)

In [ ]:
setdiff(stabilizer.stab, Perm(5))

## Stabilizer Chain

In [ ]:
import permutation: is_trivial, largest_moved_point
is_trivial(group::PermGp) = all(is_trivial, group.gens)
largest_moved_point(group::PermGp) = max(largest_moved_point.(group.gens)...)

* Schreier's theorem gives generators of the stabilizer, which can be subjected to further orbit calculations.
* Applying the above ideas along a chain of stabilizers can yield information about the whole group.

### Order of the Group

* By the **Orbit-Stabilizer Lemma**: $|G| = |x^G| \, |G_x| = |x^G| \, |y^{G_x}| \, |G_{x, y}| = {\dots}$

In [ ]:
function sizeOfGroup(group)
    is_trivial(group) && return 1
    x = largest_moved_point(group)
    orb = orbit_with_stabilizer(group.gens, x, onPoints)
    stab = PermGp(setdiff(orb.stab, group.one), group.one)
    return sizeOfGroup(stab) * length(orb.reps)
end

In [ ]:
sizeOfGroup(group)

* Some slightly bigger examples: the symmetric group on $10$ points, and the largest Mathieu group $M_{24}$:

In [ ]:
group = PermGp(transpositions(10), Perm(10))

In [ ]:
sizeOfGroup(group)

In [ ]:
m24 = PermGp([
  Perm([2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,1,24]),
  Perm([1,2,17,13,4,6,9,18,3,7,12,23,14,19,20,15,10,11,5,22,16,21,8,24]),
  Perm([24,23,12,16,18,10,20,14,21,6,17,3,22,8,19,4,11,5,15,7,9,13,2,1]),
], Perm(24))
sizeOfGroup(m24)

###  Random Element

* Random elements of a group: along the same lines as `sizeOfGroup`

In [ ]:
function randomGroupElement(group)
    is_trivial(group) && return group.one
    x = largest_moved_point(group)
    orb = orbit_with_stabilizer(group.gens, x, onPoints)
    stab = PermGp(setdiff(orb.stab, group.one), group.one)
    return randomGroupElement(stab) * rand(orb.reps)
end

In [ ]:
a = randomGroupElement(m24)

* Check: These random elements are **uniformly distributed**!

In [ ]:
group = PermGp(transpositions(4), Perm(4))
count = Dict(a => 0 for a in elements(group))
for i in 1:2400
    count[randomGroupElement(group)] += 1
end
count

* Also, using similar ideas:
  * **membership** test: $a \in G$?
  * express element as **word in the generators**: $a = s_1 \dotsm s_k$
  * **homomorphisms** (defined on generators): $\phi(a) = \phi(s_1) \dotsm \phi(s_k)$

## Graphs

* With a further small modification, the orbit algorithm can keep track of edges and thus construct the **action graph**.  
* Recall that this is a labelled directed graph, with vertices $x, y \in X$ and edges $x \stackrel{s}{\to} y$, whenever $x.s = y$ for $x, y \in X$, and $s \in S$.  
* We need to decide on a **data structure** for such graphs.  The simplest, perhaps, is a **list of pairs** of indices, each representing an edge.

In [ ]:
using Graphs
using GraphPlot

In [ ]:
elist = [(1,2),(1,3),(1,4),(2,2)]
graph = SimpleGraph(Edge.(elist))
gplot(graph, nodelabel=vertices(graph))

* Let's turn the tree from the beginning into a simple graph and plot it.

In [ ]:
collect(enumerate(parents))

In [ ]:
graph = SimpleGraph(Edge.(enumerate(parents)))
gplot(graph, nodelabel=vertices(graph))

* The next variant of the orbit algorithm keeps track of the edges in a list `edges`.

In [ ]:
function orbit_with_edges(aaa, x, under)
    list = [x]
    edges = []
    for (i, y) in enumerate(list)
        for a in aaa
            z = under(y, a)
            l = findfirst(==(z), list)
            if isnothing(l)
                push!(list, z)
                l = length(list)
            end
            push!(edges, (i, l))
        end
    end
    return (list = list, edges = edges)
end

* Examples:

In [ ]:
elist = orbit_with_edges(swaps, 1, onPoints).edges

In [ ]:
graph = SimpleGraph(Edge.(elist))
gplot(graph, nodelabel=vertices(graph))

* The action of $G$ on $X$ induces an action on the pairs $(x, y) \in X \times X$: $((x, y), a) \mapsto (x.a, y.a)$.
* We can define a corresponding action function `onPairs`.

<div class="alert alert-warning">

<img src="images/julia.png" alt="julia" width="30">  

* [Pair](https://docs.julialang.org/en/v1/base/collections/#Core.Pair)
* [Set](https://docs.julialang.org/en/v1/base/collections/#Base.Set)
</div>

In [ ]:
onPairs(pair::Pair, a::Perm) = pair[1]^a => pair[2]^a

In [ ]:
elist = orbit_with_edges(swaps, 1 => 2, onPairs).edges
elist = filter(x -> x[1] != x[2], elist)

In [ ]:
graph = SimpleGraph(Edge.(elist))
gplot(graph, nodelabel=vertices(graph))

* The action of $G$ on $X$ induces an **action on the subsets** $Y \in 2^X$: $(Y, a) \mapsto Y^a = \{y^a : y \in Y\}$.
* We can implement a corresponding action function `onSets`.

In [ ]:
onSets(set::Set, a ::Perm) = Set([x^a for x in set])

In [ ]:
elist = orbit_with_edges(transpositions(6), Set([1,2]), onSets).edges
elist = filter(x -> x[1] != x[2], elist)

In [ ]:
graph = SimpleGraph(Edge.(elist))
gplot(graph, nodelabel=vertices(graph))

* Finally, we construct the (Cayley) graph of all permutations of $4$ points.

In [ ]:
elist = orbit_with_edges(transpositions(4), Perm(4), onRight).edges
elist = filter(x -> x[1] != x[2], elist)

In [ ]:
graph = SimpleGraph(Edge.(elist))
gplot(graph, nodelabel=vertices(graph))

### Permutations

* The action graph is in fact a directed graph.
* It encodes the permutations induced by the action on the domain.
* For this (and other applications) it will be more convenient to store the edge information in a different format: as lists of `images`.

In [ ]:
function orbit_with_images(aaa, x, under)
    list = [x]
    images = [Int[] for a in aaa]
    for (i, y) in enumerate(list)
        for (k, a) in enumerate(aaa)
            z = under(y, a)
            l = findfirst(==(z), list)
            if l == nothing
                push!(list, z);
                l = length(list);
            end
            push!(images[k], l)
        end
    end
    return (list = list, images = images)
end

* For example:

In [ ]:
swaps = transpositions(5)
orb = orbit_with_images(transpositions(5), 1, onPoints)

* Convert image lists into permutations.

In [ ]:
Perm.(orb.images)

* A more interesting example:

In [ ]:
orb = orbit_with_images(swaps, Set([1,2]), onSets)
perms = Perm.(orb.images)

In [ ]:
for perm in perms
    println(cycles(perm))
end

* When the images are converted into edges, it helps to be able to list entries together with their position in a list.  This can be regarded as the transpose of the usual two-line notation for permutations.

In [ ]:
collect(enumerate(orb.images[1]))

In [ ]:
elist = union(enumerate.(orb.images)...)
elist = [(x, y) for (x, y) in elist if x != y]
graph = SimpleGraph(Edge.(elist))
gplot(graph, nodelabel=vertices(graph))

## Exercises

* Is Euclid's Algorithm BFS or DFS?  What is the tree behind it, vertices, edges?  Make the tree explicit and use it to formulate the [Extended Euclidean Algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm), which expresses the gcd $d$ of $a$ and $b$ as a linear combination $d = ax + by$.

* **Graph Traversal.**  Modify BFS and DFS so that it can be used to visit the nodes of a (connected, simple) graph, taking care of repeat encounters.

* Apply your algorithms to this graph ... and vertex 1 ...

* **Pólya Action**.  The symmetric group of degree $n$ acts on the words of length $n$ (over a finite alphabet $A$) by rearranging the letters of a given word.  In GAP, this action is called `Permuted`.  Compute the orbit of the word `"11222"` under the action, find the permutations induced by the action on the list of all words in the orbit, plot the action graph and determine the stabilizer of the word.

In [ ]:
word = [1,1,2,2,2]
swaps = transpositions(length(word))
orb = orbit_with_images(swaps, word, permuted)
stab = orbit_with_stabilizer(swaps, word, permuted)
orb.list

*  **Conjugacy Classes.** Show that the partition of a (finite) group into conjugacy classes can be computed as an orbit under right multiplication, provided that the set of generators is closed under conjugation.

* **Subgroups.** For a finite group $G$, show that the monoid $(2^G, \cup)$ acts on the set of subgroups $H$ of $G$ via **closure**: $H.A = \langle H, A\rangle$, and that each subgroup of $G$ lies in the orbit of the trivial subgroup under this action.

* Show that $(a, n) \mapsto a^n$ does not define an action of the (additive) group of the integers $\mathbb{Z}$ on a finite group $G$. However, for any $x \in G$, the map $(a, n) \mapsto a x^n$ does define an action of $\mathbb{Z}$ on $G$.

* The **zuppos** of a finite group $G$ are the cyclic subgroups of prime power order (**z**yklische **U**ntergruppen von **P**rimzahl**p**otenz**o**rdnung).  Let $Z \subseteq G$ be a set that contains exactly one generator of each zuppo.  Describe and implement an algorithm that constructs $Z$.  Show that the singletons $\{z\}$, $z \in Z$, suffice as generators to find all subgroups of $G$ under the closure action of $2^G$.

* Show that the conjugacy classes of subgroups of a finite group can be computed as an orbit under the action of its power set.

* Prove Schreier's Theorem.

* Show that the power set $2^A = \{B : B \subseteq A\}$ of a (finite) set $A$ is the orbit of $A$ under the take-away action:
```julia
takeAway(set, s) = setdiff(set, s)
```

In [ ]:
using syt
set = Set("pizza")
orbit(set, set, takeAway)

* Describe other interesting problems, where the right choice of action turns the solution into an application of the orbit algorithm.

* A **composition** of $n$ is a sequence $c = (c_1, \dots, c_k)$ of positive integers.  Set up a bijection between the set of compositions of $n$ and the power set of $A = \{1,\dots, n{-}1\}$ (so that $A$ corresponds to the composition $(n)$, and the subset relationship on $2^A$ corresponds to refinement of compositions).  Use the bijective correspondence to compute the set of all compositions of $n$ as orbit of $(n)$ under a suitable take-away action.

* A **partition** of $n$ is a composition $\lambda = (l_1, \dots, l_k)$ where $l_1 \geq \dots \geq l_k$.  Thus, sorting the parts of any composition in decreasing order yields a partition.  In that sense, a partition is a canonical representative of a rearrangement class of compositions.  Compute the partitions of $n$ as orbit of the partition $(n)$ under a suitable action on canonical composition rearrangement class representatives.

In [ ]:
using syt
p4 = partitions(4)

* Formulate a version of BFS that, for a given vertex $x$ in a simple connected graph $\Gamma$, finds a **shortest path** to any vertex $y$ in $\Gamma$.

* Formulate an algorithm that, for given vertex in a simple connected graph $\Gamma$, finds **all** shortest paths to any vertex $y$ in $\Gamma$.

* Say that a partition $\lambda$ **covers** a partition $\mu$ if $\mu$ can be obtained by decreasing a part of $\lambda$ by $1$.  Denote by $\geq$ the reflexive and transitive closure of the covering relation.  The **Young lattice** $Y(\lambda)$ is the graph on all partitions $\mu \leq \lambda$ with the covering relation as edges.  For a partition $\lambda$, compute its Young lattice $Y(\lambda)$ as orbit of $\lambda$ under a suitable action.

* A **standard Young diagram** (SYT) of shape $\lambda$ is a shortest path from the empty partition $\emptyset$ to $\lambda$ in the Young lattice $Y(\lambda)$. For a given partition $\lambda$, compute all SYTs of shape $\lambda$ as a set of shortest paths.

In [ ]:
using syt
l = standardYTs([3,1])
tableau_path(l[3])

* A **round trip** of shape $\lambda$ is a shortest path from $\emptyset$ to $\lambda$ and back along a 
(possibly different) shortest path. Verify that the total number of round trips to all the partitions $\lambda$ of $4$ is $24$.  What is the general formula for the total number of round trips for all partitions of $n$? Why?

In [ ]:
using syt
sum(length(standardYTs(x))^2 for x in partitions(4))

* Implement the **group membership test** $x \in G$, using stabilizers.

* ($*$) In practice, the number of Schreier generators of the stabilizers in the chain can grow very fast, in larger examples.  The **Schreier-Sims** algorithm intertwines orbit calculations and membership tests to keep the number of necessary generators small.  Implement such a strategy.

* ($**$) Compute the order of the Rubik's cube group
```julia
cube = SimsGp([Perm(48, cycles) for cycles in [
  [[ 1, 3, 8, 6],[ 2, 5, 7, 4],[ 9,33,25,17],[10,34,26,18],[11,35,27,19]],
  [[ 9,11,16,14],[10,13,15,12],[ 1,17,41,40],[ 4,20,44,37],[ 6,22,46,35]],
  [[17,19,24,22],[18,21,23,20],[ 6,25,43,16],[ 7,28,42,13],[ 8,30,41,11]],
  [[25,27,32,30],[26,29,31,28],[ 3,38,43,19],[ 5,36,45,21],[ 8,33,48,24]],
  [[33,35,40,38],[34,37,39,36],[ 3, 9,46,32],[ 2,12,47,29],[ 1,14,48,27]],
  [[41,43,48,46],[42,45,47,44],[14,22,30,38],[15,23,31,39],[16,24,32,40]],
]], Perm(48))
```

In [ ]:
using simsgroup
size(cube)